## Naive Bayes on Political Text

In this notebook we use Naive Bayes to explore and classify political data. See the `README.md` for full details.

In [1]:
import sqlite3
import nltk
import random
import numpy as np
from collections import Counter, defaultdict

# Feel free to include your text patterns functions
#from text_functions_solutions import clean_tokenize, get_patterns

In [2]:
convention_db = sqlite3.connect("2020_Conventions.db")
convention_cur = convention_db.cursor()

In [3]:
convention_cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(convention_cur.fetchall())

[('conventions',)]


### Part 1: Exploratory Naive Bayes

We'll first build a NB model on the convention data itself, as a way to understand what words distinguish between the two parties. This is analogous to what we did in the "Comparing Groups" class work. First, pull in the text 
for each party and prepare it for use in Naive Bayes.  

In [4]:
convention_data = []

# fill this list up with items that are themselves lists. The 
# first element in the sublist should be the cleaned and tokenized
# text in a single string. The second element should be the party. 

query_results = convention_cur.execute(
                            '''
                            SELECT text, party 
                            FROM conventions
                            ''')

for row in query_results :
    # store the results in convention_data
    convention_data.append([row[0], row[1]])

Let's look at some random entries and see if they look right. 

In [5]:
random.choices(convention_data,k=10)

[['One special memory from that trip is of a young boy I had the privilege of visiting while at Bambino Gesu Hospital in Rome, Italy. While there, I read a little boy a story and learned that he and his family had been waiting for a heart for a very long time, and he had a grim prognosis. His situation brought my staff and me to tears, and we spoke of a little ELLs as we flew to Belgium for the next part of our trip. Upon landing just a few hours later, we learned that a heart had been donated and would be going to the little one. I think about him often, along with so many amazing and strong young patients across our own country.',
  'Republican'],
 ['We’ll work to meet our extraordinary challenges because progress is made by the hopeful, not the cynical. And we will do that work together because movements are built by the many, not the few.',
  'Democratic'],
 ['I am one of a handful of people living today who have seen firsthand the immense weight and awesome power of the presidency

If that looks good, we now need to make our function to turn these into features. In my solution, I wanted to keep the number of features reasonable, so I only used words that occur at least `word_cutoff` times. Here's the code to test that if you want it. 

In [6]:
word_cutoff = 5

tokens = [w for t, p in convention_data for w in t.split()]

word_dist = nltk.FreqDist(tokens)

feature_words = set()

for word, count in word_dist.items() :
    if count > word_cutoff :
        feature_words.add(word)
        
print(f"With a word cutoff of {word_cutoff}, we have {len(feature_words)} as features in the model.")

With a word cutoff of 5, we have 2891 as features in the model.


In [9]:
def conv_features(text,fw) :
    """Given some text, this returns a dictionary holding the
       feature words.
       
       Args: 
            * text: a piece of text in a continuous string. Assumes
            text has been cleaned and case folded.
            * fw: the *feature words* that we're considering. A word 
            in `text` must be in fw in order to be returned. This 
            prevents us from considering very rarely occurring words.
        
       Returns: 
            A dictionary with the words in `text` that appear in `fw`. 
            Words are only counted once. 
            If `text` were "quick quick brown fox" and `fw` = {'quick','fox','jumps'},
            then this would return a dictionary of 
            {'quick' : True,
             'fox' :    True}
        
    """
    ret_dict = dict()
    for word in text.lower().split():
        if word in fw:
            ret_dict[word] = True
    return ret_dict

In [12]:
assert(len(feature_words) > 0)
print(list(feature_words)[:10])
print(conv_features("america is great", feature_words))
print(conv_features("the people love freedom", feature_words))

['gave', 'You’re', 'travel', 'alone', 'join', 'goodness', 'Lady', 'pray.', 'safe,', 'amendment']
{'is': True, 'great': True}
{'the': True, 'people': True, 'love': True, 'freedom': True}


Now we'll build our feature set. Out of curiosity I did a train/test split to see how accurate the classifier was, but we don't strictly need to since this analysis is exploratory. 

In [13]:
featuresets = [(conv_features(text,feature_words), party) for (text, party) in convention_data]

In [14]:
random.seed(20220507)
random.shuffle(featuresets)

test_size = 500

In [15]:
test_set, train_set = featuresets[:test_size], featuresets[test_size:]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))

0.432


In [16]:
classifier.show_most_informative_features(25)

Most Informative Features
             enforcement = True           Republ : Democr =     27.5 : 1.0
                   votes = True           Democr : Republ =     21.6 : 1.0
                 climate = True           Democr : Republ =     17.8 : 1.0
                 destroy = True           Republ : Democr =     17.1 : 1.0
                supports = True           Republ : Democr =     16.1 : 1.0
                   media = True           Republ : Democr =     15.9 : 1.0
                preserve = True           Republ : Democr =     15.1 : 1.0
                  signed = True           Republ : Democr =     15.1 : 1.0
                freedoms = True           Republ : Democr =     14.0 : 1.0
                freedom, = True           Republ : Democr =     13.4 : 1.0
                 private = True           Republ : Democr =     11.9 : 1.0
                freedom. = True           Republ : Democr =     11.5 : 1.0
                    drug = True           Republ : Democr =     10.9 : 1.0

Write a little prose here about what you see in the classifier. Anything odd or interesting?

### My Observations

I do find it interesting, but not surprising, that variations of words like freedom are listed multiple times and are heavily used by republicans. I also find it humorous that the word "liberal" is mentioned for the republicans but conservative is not mentioned here for democrats. It hints at how republicans use the word liberal as a tool in swaying their audience. Words like "preserve", "armed", and "greatness" all show the ideologies that republicans push for in their speeches as well. 

## Part 2: Classifying Congressional Tweets

In this part we apply the classifer we just built to a set of tweets by people running for congress
in 2018. These tweets are stored in the database `congressional_data.db`. That DB is funky, so I'll
give you the query I used to pull out the tweets. Note that this DB has some big tables and 
is unindexed, so the query takes a minute or two to run on my machine.

In [17]:
cong_db = sqlite3.connect("congressional_data.db")
cong_cur = cong_db.cursor()

In [18]:
results = cong_cur.execute(
        '''
           SELECT DISTINCT 
                  cd.candidate, 
                  cd.party,
                  tw.tweet_text
           FROM candidate_data cd 
           INNER JOIN tweets tw ON cd.twitter_handle = tw.handle 
               AND cd.candidate == tw.candidate 
               AND cd.district == tw.district
           WHERE cd.party in ('Republican','Democratic') 
               AND tw.tweet_text NOT LIKE '%RT%'
        ''')

results = list(results) # Just to store it, since the query is time consuming

In [19]:
tweet_data = []

for row in results:
    tweet = row[2].lower()
    party = row[1]
    tweet_data.append([tweet, party])

# Now fill up tweet_data with sublists like we did on the convention speeches.
# Note that this may take a bit of time, since we have a lot of tweets.


There are a lot of tweets here. Let's take a random sample and see how our classifer does. I'm guessing it won't be too great given the performance on the convention speeches...

In [20]:
random.seed(20201014)

tweet_data_sample = random.choices(tweet_data,k=10)

In [22]:

for tweet, party in tweet_data_sample :
    features = conv_features(tweet, feature_words)
    estimated_party = estimated_party = classifier.classify(features)
    # Fill in the right-hand side above with code that estimates the actual party
    
    print(f"Here's our (cleaned) tweet: {tweet}")
    print(f"Actual party is {party} and our classifer says {estimated_party}.")
    print("")

Here's our (cleaned) tweet: b'earlier today, i spoke on the house floor abt protecting health care for women and praised @ppmarmonte for their work on the central coast. https://t.co/wqgtrzt7vv'
Actual party is Democratic and our classifer says Democratic.

Here's our (cleaned) tweet: b'go tribe! #rallytogether https://t.co/0nxutfl9l5'
Actual party is Democratic and our classifer says Democratic.

Here's our (cleaned) tweet: b"apparently, trump thinks it's just too easy for students overwhelmed by the crushing burden of debt to pay off student loans #trumpbudget https://t.co/ckyqo5t0qh"
Actual party is Democratic and our classifer says Democratic.

Here's our (cleaned) tweet: b'we\xe2\x80\x99re grateful for our first responders, our rescue personnel, our firefighters, our police, and volunteers who have been working tirelessly to keep people safe, provide much-needed help, while putting their own lives on the line.\n\nhttps://t.co/ezpv0vmiz3'
Actual party is Republican and our classife

Now that we've looked at it some, let's score a bunch and see how we're doing.

In [25]:
# dictionary of counts by actual party and estimated party. 
# first key is actual, second is estimated
parties = ['Republican','Democratic']
results = defaultdict(lambda: defaultdict(int))

for p in parties :
    for p1 in parties :
        results[p][p1] = 0


num_to_score = 10000
random.shuffle(tweet_data)

for idx, tp in enumerate(tweet_data) :
    tweet, party = tp    
    # Now do the same thing as above, but we store the results rather
    # than printing them. 
   
    # get the estimated party
    estimated_party = classifier.classify(conv_features(tweet, feature_words))
    
    results[party][estimated_party] += 1
    
    if idx > num_to_score : 
        break

In [26]:
results

defaultdict(<function __main__.<lambda>()>,
            {'Republican': defaultdict(int,
                         {'Republican': 0, 'Democratic': 4372}),
             'Democratic': defaultdict(int,
                         {'Republican': 0, 'Democratic': 5630})})

### Reflections

The model classified all text as being democratic and did not classify any text as republican. The model is heavily biased toward democratic language and did not properly run. As shown by the code below, there is about 100,000 more tweets for democrats than republicans and is likely skewing the classifier's learning. Rebalancing the data to be even and re-running the model would help alleviate the skewness.

In [28]:
Counter([label for _, label in tweet_data])

Counter({'Democratic': 376125, 'Republican': 288531})